<a href="https://colab.research.google.com/github/IreneMuiaM/Credrails_Assesment/blob/main/ICEA_NCBA_RECON_fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install required packages
import sys

!{sys.executable} -m pip install -q ydata-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.2/356.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 22.9 MB/s eta 0:00:00


In [ ]:
# @title Imports
from pathlib import Path
from typing import Callable
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import drive
from ydata_profiling import  ProfileReport

In [ ]:
# @title Config
# file paths
BASE_DIR = Path().resolve().parent
GDRIVE_MOUNT = BASE_DIR / "gdrive"

DATA_DIR = GDRIVE_MOUNT / "MyDrive/ICEA_reconciliation/Data"
INPUT_DIR = DATA_DIR / "input"
OUTPUT_DIR = BASE_DIR / "output"

drive.mount(GDRIVE_MOUNT.as_posix(), force_remount=True)

# display
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", "{:,}".format)

# plots
sns.set_theme(context="notebook", style="whitegrid", rc={"figure.figsize": (14, 8)})

# other
ACC_NAME = "NCBA_NIC"

BANK_OS_COL_MAPPING = {
    0: "date",
    1: "transaction_details_1",
    2: "trx_id",
    3: "transaction_details_2",
    5: "amount"
}

Mounted at /gdrive


# Data ingestion and cleaning

In [ ]:
# @title Read/uploading data
bank_stmt_current = pd.read_excel(
    INPUT_DIR / "NOV BANK.xlsx",
    sheet_name="Sheet1",
    skiprows=6
)

nav_cb_current = pd.read_excel(
    INPUT_DIR / "NAV NOV.xlsx",
    sheet_name="Sheet1",
)

In [ ]:
# @title Data cleanup functions

#cleaning up the column names by removing special characters,converting them to lowercase, and replacing spaces with underscore

def clean_col_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.str.lower()
        .str.replace(r"[^\w|^\s]", "", regex=True)
        .str.replace(r"\s{2,}", " ", regex=True)
        .str.replace(r"\s", "_", regex=True)
    )
    return df

#applying a transformation function to list of columns

def transform_cols(
    df: pd.DataFrame, cols: list[str], func: Callable[[pd.Series], pd.Series]
) -> pd.DataFrame:
    df = df.copy()
    for col in cols:
        df[col] = func(df[col])
    return df

In [ ]:
# @title Applying function to clean columns
bank_stmt_current = clean_col_names(bank_stmt_current)
print(f"Bank statement(NOV):\n{bank_stmt_current.columns}")

nav_cb_current = clean_col_names(nav_cb_current)
print(f"Navision cash book(NAV):\n{nav_cb_current.columns}")

Bank statement(NOV):
Index(['date', 'transaction_details', 'value_date', 'debit', 'credit',
       'balance'],
      dtype='object')
Navision cash book(NAV):
Index(['posting_date', 'document_type', 'document_no', 'bank_account_no',
       'description', 'amount', 'amount_lcy', 'open', 'kba_bank_code',
       'kba_branch_code', 'beneficiary_account_no', 'beneficiary_account_name',
       'swift_code', 'payment_mode', 'eft_file_name', 'paid',
       'eft_file_regenerated', 'entry_no', 'reversal_reason', 'description_2',
       'subsystem_doctype', 'navpayingbankcategory', 'um_username',
       'busledger_type_code', 'policy_no', 'ilmsreqbatchno', 'same'],
      dtype='object')


In [ ]:
# @title Cleaning the bank statement

#filter rows with unamed columns
bank_stmt_current_clean = (
    bank_stmt_current.filter(regex=r"^((?!unnamed).*)$")

    # Cleaning date cols
    .pipe(
        lambda df: transform_cols(
            df=df,
            cols=df.filter(regex="(?i)date").columns.to_list(),
            func=lambda s: pd.to_datetime(s, errors="coerce", format="%d%m%Y").fillna(
                pd.to_datetime(s, errors="coerce")
            )
        )
    )
    .assign(
        transaction_details_1=lambda df: df["transaction_details"],
        transaction_details_2=lambda df: df["transaction_details"].shift(-1),
        debit=lambda df: df["debit"] * -1,
        amount=lambda df: df["debit"].fillna(df["credit"]),

    )
    .drop(columns=["transaction_details", "debit", "credit"])
    .dropna(subset=["date", "amount"])
    .reset_index(drop=True)
)
bank_stmt_current_clean.info()
display(bank_stmt_current_clean.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172 entries, 0 to 3171
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   3172 non-null   datetime64[ns]
 1   value_date             3172 non-null   datetime64[ns]
 2   balance                3172 non-null   float64       
 3   transaction_details_1  3172 non-null   object        
 4   transaction_details_2  3172 non-null   object        
 5   amount                 3172 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(2)
memory usage: 148.8+ KB


,date,value_date,balance,transaction_details_1,transaction_details_2,amount
0,2023-11-01,2023-11-01,"76,038,031.8",Standing Order FT233054RNZQ,POLICY PREMIUM 029/VES/155738 EUNICE KAMBA MBURUGU,"2,206.0"
1,2023-11-01,2023-11-01,"76,046,051.8",Standing Order FT23305LCPC5,093715/093716/143362/171223 MARTIN KIMANI KANGETHE,"8,020.0"
2,2023-11-01,2023-11-01,"76,110,051.8",Standing Order FT23305RQKBY,JK Policy no. 027/EAW/232465 JANE JEMATIA KIPLAGAT,"64,000.0"
3,2023-11-01,2023-11-01,"76,120,076.8",Standing Order FT23305K8ZVX,025/NCBAEDU/267594 PURITY GAKII,"10,025.0"
4,2023-11-01,2023-11-01,"76,122,081.8",Standing Order FT2330502XLT,POLICY NO: 029/EAW/208751 INVESTMENT POLICY MARGARET WACHIRA WAIHUINI,"2,005.0"


In [ ]:
def parse_trx_details(df: pd.DataFrame) -> pd.DataFrame:
    # Extracting Bulk Transfer Numbers:
    bulk_transfer_numbers = (
        df["trx_detail"].pipe(lambda s: s.where(s.str.contains("IB Bulk")))
        .str.split(pat=r"Transfer|BKM\d+", expand=True).get(1)
    )
    if bulk_transfer_numbers is not None:
        bulk_transfer_numbers = (
            bulk_transfer_numbers.str.replace(r"[^\w|\s]", "", regex=True)
            .rename("bulk_trans_id")
            .str.strip()
        )
    else:
        bulk_transfer_numbers = pd.Series(name="bulk_trans_id")

    # other IDs
    patterns = (
        r"(?P<ft_no>FT\w{10,11})", r"(?P<tt_no>TT\w{10,11})",
        r"(?P<aaa_no>AAA\w{15})", r"(?P<md_no>MD\w{10,11})",
        r"(?P<charge_id>CHG\w{9})", r"(?P<batch_no>[A-Z]{3}\d{12})",
        r"(?i)inhouse cheque\s(?P<inhouse_cheque_no>\d+)",
        r"(?i)cheque\s(?:deposit\s)?(?:CHQ)?(?:\sNO)?(?:-)?(?P<cheque_no>\d{6})",
    )
    features = []
    for pat in patterns:
        features.append(df["trx_detail"].str.extract(pat))

    # results
    features_df = pd.concat(features, axis=1)
    return features_df.join(bulk_transfer_numbers)


def classify_bank_transaction(df: pd.DataFrame) -> np.ndarray:
    conditions = [
        df["amount"].lt(0),
        df["amount"].gt(0)
    ]
    choices = ["debit", "credit"]
    return np.select(conditions, choices, default="undefined")


def is_bank_charge(df: pd.DataFrame) -> pd.Series:
    df = df.copy()
    explicit = df["trx_detail"].str.contains(
        r"(?i)charge|ledger fees|duty|stamp", regex=True
    )
    df["abs_amount"] = df["amount"].abs()
    implicit = (
        df["abs_amount"] / df.groupby("batch_no")["abs_amount"].transform("sum")
    ) < 0.02
    return df["charge_id"].notna() | explicit | implicit


def classify_cash_book_entry(df: pd.DataFrame) -> np.ndarray:
    conditions = [
        df["document_no"].str.fullmatch(r"\d+") | df["document_no"].str.fullmatch(r"FR-\w+"),
        df["document_no"].str.fullmatch(r"FPV-\w+"),
        df["document_no"].str.fullmatch(r"G\d+") | df["document_no"].str.fullmatch(r"JV.+")
    ]
    choices = ["receipt", "payment", "general voucher"]
    return np.select(conditions, choices, default="undefined")

In [ ]:
# @title Preprocess bank statement
NCBA_REQUIRED_COLS = [
    "date",
    "transaction_details_1",
    "transaction_details_2",
    "amount",
]
bank_df = (
    bank_stmt_current_clean[NCBA_REQUIRED_COLS].assign(
        trx_type=lambda df: classify_bank_transaction(df),
        trans_type=lambda df: df["transaction_details_1"].str.replace(
            r"\s(FT|MD)\w+", "", regex=True
        ),
        trx_detail=lambda df: (
            df[["transaction_details_1", "transaction_details_2"]].fillna("")
            .astype("str")
            .agg(" ".join, axis=1)
            .str.strip()
            .replace("", None)
        ),
        period=lambda df: df["date"].dt.to_period(freq="Q"),
    )
    .pipe(lambda df: df.join(parse_trx_details(df)))
    .assign(is_charge=lambda df: is_bank_charge(df))
    .reset_index(names="index_bank")
)
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172 entries, 0 to 3171
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   index_bank             3172 non-null   int64         
 1   date                   3172 non-null   datetime64[ns]
 2   transaction_details_1  3172 non-null   object        
 3   transaction_details_2  3172 non-null   object        
 4   amount                 3172 non-null   float64       
 5   trx_type               3172 non-null   object        
 6   trans_type             3172 non-null   object        
 7   trx_detail             3172 non-null   object        
 8   period                 3172 non-null   period[Q-DEC] 
 9   ft_no                  2760 non-null   object        
 10  tt_no                  121 non-null    object        
 11  aaa_no                 268 non-null    object        
 12  md_no                  19 non-null     object        
 13  cha

In [ ]:
# Combine the transaction id from tt, aa, and md

def merge_trans_id(df: pd.DataFrame) ->pd.DataFrame:
  df=df.copy()
  df['ft_no']=df.apply(lambda row : row["ft_no"] if pd.notna(row["ft_no"]) \
                       else (row["tt_no"] if pd.notna(row["tt_no"]) \
                       else (row["aaa_no"] if pd.notna(row["aaa_no"]) \
                       else row["md_no"] )),axis=1 )
  df=df.drop(columns=["tt_no","aaa_no","md_no"])
  return df.rename(columns={'ft_no':"merged_trans_id"})

def batch_bulk(df: pd.DataFrame) ->pd.DataFrame:
  df=df.copy()
  df["batch_bulk_id"] = df.apply(lambda row : row["batch_no"] if pd.notna(row["batch_no"]) \
                       else (row["bulk_trans_id"] if pd.notna(row["bulk_trans_id"]) \
                       else row["batch_no"]
                       ),axis=1)
  return df


In [ ]:
def extract_and_remove_patterns(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    # Define the patterns to be extracted
    patterns = [
        r"\d{3}/[A-Za-z]{3}/\d+",  # e.g., "123/ABC/456"
        r"\d{3}/[A-Za-z]{3}\d+",   # e.g., "123/ABC456"
        r"\d{3}[A-Za-z]{3}/\d+",   # e.g., "123ABC/456"
        r"\d{3}[A-Za-z]{3}\d+",    # e.g., "123ABC456"
        r"\d{3}[A-Za-z]{3} \d+",   # e.g., "123ABC 456"
        r"\d{3} [A-Za-z]{3}\d+",   # e.g., "123 ABC456"
        r"\d{3} [A-Za-z]{3} \d+",  # e.g., "123 ABC 456"
        r"\d{3}/'NCBAEDU/\d+" ,   # e.g., "123/'NCBAEDU'/456"
        r"\d{3}/NCBAEDU\d+",     # e.g., "123/'NCBAEDU'456"
        r"\d{3}NCBAEDU\d+",      # e.g., "123'NCBAEDU'/456"
        r"\d{3} NCBAEDU \d+"    # e.g., "123 'NCBAEDU' 456"
        ]


      # Combine the patterns into a single regex with alternation
    combined_pattern = "|".join(patterns)

    # Function to extract the pattern and remove it from the original text
    def extract_pattern(text):
        match = re.search(combined_pattern, text, re.IGNORECASE)  # Case-insensitive search
        if match:
            extracted = match.group()
            cleaned_text = re.sub(combined_pattern, "", text, flags=re.IGNORECASE).strip()  # Case-insensitive removal
            return extracted, cleaned_text
        return None, text

    # Apply the extraction and removal function
    df[['policy_extracted', column_name]] = df[column_name].apply(
        lambda x: pd.Series(extract_pattern(str(x)))
    )

    return df

In [ ]:

def clean_column1(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    # Function to clean text by removing spaces and special characters and converting to lowercase
    def clean_text(text):
      if pd.isna(text):  # Check for NaN
            return np.nan
      if text is None:  # Check for None
            return None
      # Remove spaces and special characters
      cleaned_text = re.sub(r'[^A-Za-z0-9]', '', text)
      # Convert to lowercase
      cleaned_text = cleaned_text.lower()

      return cleaned_text

    # Apply the clean_text function to the specified column
    df[column_name] = df[column_name].apply(lambda x: clean_text(str(x)))

    return df

def clean_column2(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    # Define a function to clean the text
    def clean_text(text: str) -> str:
        if pd.isna(text):  # Check for NaN
            return np.nan
        elif text is None:  # Check for None
            return None
        else:
        # Remove special characters (keep only letters, numbers, and spaces)
             cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', text)
        # Replace multiple spaces with a single space
             cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        # Convert to lowercase
             cleaned_text = cleaned_text.lower().strip()
        return cleaned_text

    # Apply the cleaning function to the specified column
    df[column_name] = df[column_name].apply(lambda x: clean_text(str(x)))

    return df

In [ ]:

bank_df2=merge_trans_id(bank_df)
bank_df2=batch_bulk(bank_df2)
bank_df3 = extract_and_remove_patterns(bank_df2, 'transaction_details_2')
bank_df4=clean_column1(bank_df3,"policy_extracted")
bank_df4=clean_column2(bank_df4,"transaction_details_2")
bank_df4=clean_column2(bank_df4,"batch_bulk_id")

In [ ]:
bank_df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172 entries, 0 to 3171
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   index_bank             3172 non-null   int64         
 1   date                   3172 non-null   datetime64[ns]
 2   transaction_details_1  3172 non-null   object        
 3   transaction_details_2  3172 non-null   object        
 4   amount                 3172 non-null   float64       
 5   trx_type               3172 non-null   object        
 6   trans_type             3172 non-null   object        
 7   trx_detail             3172 non-null   object        
 8   period                 3172 non-null   period[Q-DEC] 
 9   merged_trans_id        3168 non-null   object        
 10  charge_id              237 non-null    object        
 11  batch_no               264 non-null    object        
 12  inhouse_cheque_no      30 non-null     object        
 13  che

In [ ]:
# @title Preprocess Navision cash book
# exclude  "unnamed" columns from the DataFrame
nav_cash_book = nav_cb_current.filter(regex=r"^((?!unnamed).*)$")
nav_cash_book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20887 entries, 0 to 20886
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   posting_date              20887 non-null  datetime64[ns]
 1   document_type             0 non-null      float64       
 2   document_no               20887 non-null  object        
 3   bank_account_no           20887 non-null  object        
 4   description               20885 non-null  object        
 5   amount                    20887 non-null  float64       
 6   amount_lcy                20887 non-null  float64       
 7   open                      20887 non-null  bool          
 8   kba_bank_code             20059 non-null  float64       
 9   kba_branch_code           19978 non-null  float64       
 10  beneficiary_account_no    2788 non-null   object        
 11  beneficiary_account_name  20377 non-null  object        
 12  swift_code        

In [ ]:
# filter required columns
NAV_CB_REQUIRED_COLS = [
    "entry_no",
    "posting_date",
    "amount",
    "eft_file_name",
    "document_no",
    "policy_no",
    "beneficiary_account_name",
    "description",
   # "ilmsreqbatchno",
    # "busledger_type_code",
    "bank_account_no",
    # "payment_mode",
]
nav_cb_df = (
    nav_cash_book[NAV_CB_REQUIRED_COLS]
    .assign(trx_type=lambda df: classify_cash_book_entry(df))
    .reset_index(names="index_cb")
)

In [ ]:

nav_cb_df=clean_column1(nav_cb_df,"policy_no")
nav_cb_df=clean_column2(nav_cb_df,"beneficiary_account_name")
nav_cb_df["Nav_batch_bulk_id"]=nav_cb_df["eft_file_name"].str.lower()
nav_cb_df.head()

,index_cb,entry_no,posting_date,amount,eft_file_name,document_no,policy_no,beneficiary_account_name,description,bank_account_no,trx_type,Nav_batch_bulk_id
0,0,28533671,2023-11-01,"-1,500,000.0",INS103331011123,FPV-INS2023005697,nan,advance cars kenya limited,KDN530F DENIS ODOYO,2-740-02-002,payment,ins103331011123
1,1,28533673,2023-11-01,"-45,000.0",RK163LN22A,2023020025786,027aen107082,kangau james njenga,NIC HEAD OFFICE,2-740-02-002,receipt,rk163ln22a
2,2,28533687,2023-11-01,"-59,182.2",INS103017011123,2023020025493,028aen178027,elizabeth a awitty,NIC HEAD OFFICE,2-740-02-002,receipt,ins103017011123
3,3,28533690,2023-11-01,"-817,909.94",INS103017011123,2023020025294,028eaw073607,thomas odhiambo abade,NIC HEAD OFFICE,2-740-02-002,receipt,ins103017011123
4,4,28533692,2023-11-01,"-730,680.88",INS103017011123,2023020025720,022cea078232,jeremiah apalia lomari,NIC HEAD OFFICE,2-740-02-002,receipt,ins103017011123


In [ ]:
bank_df4.head(2)

,index_bank,date,transaction_details_1,transaction_details_2,amount,trx_type,trans_type,trx_detail,period,merged_trans_id,charge_id,batch_no,inhouse_cheque_no,cheque_no,bulk_trans_id,is_charge,batch_bulk_id,policy_extracted
0,0,2023-11-01,Standing Order FT233054RNZQ,policy premium eunice kamba mburugu,"2,206.0",credit,Standing Order,Standing Order FT233054RNZQ POLICY PREMIUM 029/VES/155738 EUNICE KAMBA MBURUGU,2023Q4,FT233054RNZQ,NaN,NaN,NaN,NaN,None,False,nan,029ves155738
1,1,2023-11-01,Standing Order FT23305LCPC5,093715093716143362171223 martin kimani kangethe,"8,020.0",credit,Standing Order,Standing Order FT23305LCPC5 093715/093716/143362/171223 MARTIN KIMANI KANGETHE,2023Q4,FT23305LCPC5,NaN,NaN,NaN,NaN,None,False,nan,none


# Reconciliation with record linkage



In [ ]:
# @title preparing data for recon

#standardize columnnames for NOV_bank
def standard_b(df : pd.DataFrame) -> pd.DataFrame:
  df=df.rename(columns={"index_bank" : "unique_id","merged_trans_id":"trans_id","date" : "trx_date","transaction_details_2":"trans_detail",\
                        "policy_extracted":"policy_no","amount":"trx_amount","batch_bulk_id":"m_batch_bulk_id"})
  df=df.drop(columns=["period"])
  return df
#standardize columnnames for NAV_Cashbook
def standard_c(df : pd.DataFrame) -> pd.DataFrame:
  df=df.rename(columns={"index_cb" : "unique_id","eft_file_name":"trans_id","posting_date" : "trx_date","beneficiary_account_name":"trans_detail",\
                        "policy_no":"policy_no","amount":"trx_amount","Nav_batch_bulk_id":"m_batch_bulk_id"})
  return df

# Apply functions
bank_df4_splink= standard_b(bank_df4)
nav_cb_df_splink= standard_c(nav_cb_df)

In [ ]:
# @title Install splink
!pip install splink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


In [ ]:
!pip install duckdb

In [ ]:
!pip install jellyfish

In [ ]:
from splink import DuckDBAPI
db_api = DuckDBAPI()
from splink.exploratory import profile_columns

profile_columns(bank_df4_splink, db_api=DuckDBAPI(), top_n=10, bottom_n=5)

alt.VConcatChart(...)

In [ ]:
# @title Testing blocking rules

from splink import DuckDBAPI, block_on
from splink.blocking_analysis import count_comparisons_from_blocking_rule


db_api = DuckDBAPI()

br0 = block_on("trans_id")
br1 = block_on("trx_date", "trans_id")
br2 = block_on("trans_detail", "trx_amount")
br3 = block_on( "policy_no")
br4 = block_on( "m_batch_bulk_id")

counts0 = count_comparisons_from_blocking_rule(
    table_or_tables= [bank_df4_splink, nav_cb_df_splink],
    blocking_rule=br0,
    link_type="link_only",
    db_api=db_api,
)
print(counts0)

counts1 = count_comparisons_from_blocking_rule(
    table_or_tables= [bank_df4_splink, nav_cb_df_splink],
    blocking_rule=br1,
    link_type="link_only",
    db_api=db_api,
)
print(counts1)

counts2= count_comparisons_from_blocking_rule(
    table_or_tables= [bank_df4_splink, nav_cb_df_splink],
    blocking_rule=br2,
    link_type="link_only",
    db_api=db_api,
)
print(counts2)

counts3= count_comparisons_from_blocking_rule(
    table_or_tables= [bank_df4_splink, nav_cb_df_splink],
    blocking_rule=br3,
    link_type="link_only",
    db_api=db_api,
)
print(counts3)


counts4= count_comparisons_from_blocking_rule(
    table_or_tables= [bank_df4_splink, nav_cb_df_splink],
    blocking_rule=br4,
    link_type="link_only",
    db_api=db_api,
)
print(counts4)

{'number_of_comparisons_generated_pre_filter_conditions': 8426, 'number_of_comparisons_to_be_scored_post_filter_conditions': 8426, 'filter_conditions_identified': '', 'equi_join_conditions_identified': 'l."trans_id" = r."trans_id"', 'link_type_join_condition': 'where l."source_dataset" || \'-__-\' || l."unique_id" < r."source_dataset" || \'-__-\' || r."unique_id" and l."source_dataset" != r."source_dataset"'}
{'number_of_comparisons_generated_pre_filter_conditions': 4934, 'number_of_comparisons_to_be_scored_post_filter_conditions': 4934, 'filter_conditions_identified': '', 'equi_join_conditions_identified': 'l."trx_date" = r."trx_date" AND l."trans_id" = r."trans_id"', 'link_type_join_condition': 'where l."source_dataset" || \'-__-\' || l."unique_id" < r."source_dataset" || \'-__-\' || r."unique_id" and l."source_dataset" != r."source_dataset"'}
{'number_of_comparisons_generated_pre_filter_conditions': 4, 'number_of_comparisons_to_be_scored_post_filter_conditions': 4, 'filter_condition

In [ ]:
from splink.blocking_analysis import n_largest_blocks

result = n_largest_blocks(table_or_tables=[bank_df4_splink, nav_cb_df_splink],
    blocking_rule= block_on("trans_id", "trx_date"),
    link_type="link_only",
    db_api=db_api,
    n_largest=5
    )

result.as_pandas_dataframe()

,key_0,key_1,count_l,count_r,block_count
0,FT23332J7TZ4,2023-11-28,1,1105,1105
1,FT233142C2JF,2023-11-10,1,676,676
2,FT23332JJ63W,2023-11-28,1,431,431
3,FT23319XVJPG,2023-11-15,1,304,304
4,FT23319WQXD7,2023-11-15,1,187,187


In [ ]:
bank_df4_splink1=bank_df4_splink[['unique_id', 'trx_date','trans_detail','trx_amount','trans_id','policy_no',"m_batch_bulk_id"]]
nav_cb_df_splink1=nav_cb_df_splink[['unique_id', 'trx_date','trans_detail','trx_amount','trans_id','policy_no',"m_batch_bulk_id"]]

In [ ]:
print(bank_df4_splink1.info())
print(nav_cb_df_splink1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172 entries, 0 to 3171
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   unique_id        3172 non-null   int64         
 1   trx_date         3172 non-null   datetime64[ns]
 2   trans_detail     3172 non-null   object        
 3   trx_amount       3172 non-null   float64       
 4   trans_id         3168 non-null   object        
 5   policy_no        3172 non-null   object        
 6   m_batch_bulk_id  3172 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 173.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20887 entries, 0 to 20886
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   unique_id        20887 non-null  int64         
 1   trx_date         20887 non-null  datetime64[ns]
 2   tran

In [ ]:
from jellyfish import levenshtein_distance # Import levenshtein_distance

from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
)

blocking_rules_for_analysis = [
    block_on("trans_id", "trx_date"),
    block_on("trans_detail"),
    block_on("policy_no","trx_date"),
    #block_on("trx_amount")
    #block_on("trans_id", "trans_detail"),
    block_on("m_batch_bulk_id")
]


cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=[bank_df4_splink1, nav_cb_df_splink1],
    blocking_rules=blocking_rules_for_analysis,
    db_api=db_api,
    link_type="link_only",
)

alt.Chart(...)

In [ ]:
# @title Blocking rule settings
# Define the settings
from splink import Linker, SettingsCreator, block_on, DuckDBAPI
import splink.comparison_level_library as cll
import splink.comparison_library as cl

detailed_model_settings = SettingsCreator(
    unique_id_column_name="unique_id",
    link_type="link_only",
    blocking_rules_to_generate_predictions=blocking_rules_for_analysis,
    comparisons=[
        cl.NameComparison("trans_id"),
        cl.NameComparison("m_batch_bulk_id"),
        cl.NameComparison("policy_no").configure(term_frequency_adjustments=True),
        cl.DateOfBirthComparison("trx_date",
            input_is_string=True,
            datetime_format="%Y%m%d",
            invalid_dates_as_null=True,
        ),
        #cl.DamerauLevenshteinAtThresholds("trans_detail", [1, 2]),
        cl.ExactMatch("trx_amount").configure(term_frequency_adjustments=True),
        cl.DamerauLevenshteinAtThresholds("trans_detail", [1, 2]).configure(
            term_frequency_adjustments=True
        ),

    ],
    retain_intermediate_calculation_columns=True,
)
# Explicitly cast the 'trx_date' column to VARCHAR before passing it to the Linker
bank_df4_splink1['trx_date'] = bank_df4_splink1['trx_date'].astype(str)
nav_cb_df_splink1['trx_date'] = nav_cb_df_splink1['trx_date'].astype(str)

linker_detailed = Linker([bank_df4_splink1, nav_cb_df_splink1], detailed_model_settings, db_api=DuckDBAPI())


<ipython-input-59-8ed4219dbe53>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_df4_splink1['trx_date'] = bank_df4_splink1['trx_date'].astype(str)
<ipython-input-59-8ed4219dbe53>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nav_cb_df_splink1['trx_date'] = nav_cb_df_splink1['trx_date'].astype(str)


In [ ]:
# @title Estimation of probability_two_random_records_match
#In some cases, the probability_two_random_records_match will be known.
#For example, if you are linking two tables of 10,000 records and expect
# a one-to-one match, then you should set this value to 1/10_000 in your settings instead of estimating it.

deterministic_rules = [
    block_on("trans_id", "trx_date"),
    #block_on("policy_no"),
    block_on("m_batch_bulk_id","trx_date")
]

linker_detailed.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

INFO:splink.internals.linker_components.training:Probability two random records match is estimated to be  0.000137.
This means that amongst all possible pairwise record comparisons, one in 7,287.48 are expected to match.  With 66,253,564 total possible comparisons, we expect a total of around 9,091.43 matching pairs


In [ ]:
# @title Estimation of u probabilities¶

linker_detailed.training.estimate_u_using_random_sampling(max_pairs=1e6)

INFO:splink.internals.estimate_u:----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

INFO:splink.internals.m_u_records_to_parameters:u probability not trained for trx_date - Exact match on date of birth (comparison vector value: 5). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for trx_date - DamerauLevenshtein distance <= 1 (comparison vector value: 4). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for trx_date - Abs date difference <= 1 month (comparison vector value: 3). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for trx_date - Abs date difference <= 1 year (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for

In [ ]:
# @title Estimation of probability_two_random_records_match -m - probabilities
#In some cases, the probability_two_random_records_match will be known. For example, if you are linking two tables of 10,000 records and expect a one-to-one match, then you should set this value to 1/10_000 in your settings instead of estimating it.
training_blocking_rule = block_on("trans_id", "trx_amount")
training_session_fname_sname = (
    linker_detailed.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)

INFO:splink.internals.em_training_session:
----- Starting EM training session -----

INFO:splink.internals.em_training_session:Estimating the m probabilities of the model by blocking on:
(l."trans_id" = r."trans_id") AND (l."trx_amount" = r."trx_amount")

Parameter estimates will be made for the following comparison(s):
    - m_batch_bulk_id
    - policy_no
    - trx_date
    - trans_detail

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - trans_id
    - trx_amount
INFO:splink.internals.expectation_maximisation:
Level Exact match on m_batch_bulk_id on comparison m_batch_bulk_id not observed in dataset, unable to train m value

Level Jaro-Winkler distance of m_batch_bulk_id >= 0.92 on comparison m_batch_bulk_id not observed in dataset, unable to train m value

Level Jaro-Winkler distance of m_batch_bulk_id >= 0.88 on comparison m_batch_bulk_id not observed in dataset, unable to train m value

Level Jaro-Winkler distance

In [ ]:
training_blocking_rule = block_on("m_batch_bulk_id","trx_date")
training_session_policy = linker_detailed.training.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)

INFO:splink.internals.em_training_session:
----- Starting EM training session -----

INFO:splink.internals.em_training_session:Estimating the m probabilities of the model by blocking on:
(l."m_batch_bulk_id" = r."m_batch_bulk_id") AND (l."trx_date" = r."trx_date")

Parameter estimates will be made for the following comparison(s):
    - trans_id
    - policy_no
    - trx_amount
    - trans_detail

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - m_batch_bulk_id
    - trx_date
INFO:splink.internals.expectation_maximisation:
Level Exact match on trans_id on comparison trans_id not observed in dataset, unable to train m value

Level Jaro-Winkler distance of trans_id >= 0.92 on comparison trans_id not observed in dataset, unable to train m value

Level Jaro-Winkler distance of trans_id >= 0.88 on comparison trans_id not observed in dataset, unable to train m value

Level Jaro-Winkler distance of trans_id >= 0.7 on compariso

In [ ]:
## @title Visualization

linker_detailed.visualisations.match_weights_chart()


alt.VConcatChart(...)

In [ ]:
linker_detailed.visualisations.m_u_parameters_chart()

alt.HConcatChart(...)

In [ ]:
settings = linker_detailed.misc.save_model_to_json(
    "../saved_model_from_demo.json", overwrite=True)

In [ ]:

predictions = linker_detailed.inference.predict(threshold_match_probability=0.5)
df_predictions = predictions.as_pandas_dataframe()
df_predictions.head(5)

INFO:splink.internals.linker_components.inference:Blocking time: 0.20 seconds


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

INFO:splink.internals.linker_components.inference:Predict time: 49.41 seconds
 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'trans_id':
    m values not fully trained
Comparison: 'm_batch_bulk_id':
    m values not fully trained
Comparison: 'policy_no':
    m values not fully trained
Comparison: 'trx_date':
    m values not fully trained
Comparison: 'trx_date':
    u values not fully trained
Comparison: 'trans_detail':
    m values not fully trained
Comparison: 'trans_detail':
    u values not fully trained


,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,trans_id_l,trans_id_r,gamma_trans_id,tf_trans_id_l,...,bf_trx_amount,bf_tf_adj_trx_amount,trans_detail_l,trans_detail_r,gamma_trans_detail,tf_trans_detail_l,tf_trans_detail_r,bf_trans_detail,bf_tf_adj_trans_detail,match_key
0,7.310427492261469,0.9937394201129508,__splink__input_table_0,__splink__input_table_1,91,3572,AAACT23305450LQHZW,AAACT23305450LQHZW,4,8.337849668570476e-05,...,1.85241107447845,19.570679023309708,account aaact23305450lqhzw,ncba bank,0,4.1564487302049126e-05,0.0005819028222286878,1.0,1.0,0
1,0.5787988699638531,0.5989742105243553,__splink__input_table_0,__splink__input_table_1,165,5172,TT23306DFBL3,TT23306DFBL3,4,8.337849668570476e-05,...,1.85241107447845,0.920973130508692,tt23306dfbl3 rosemary wanjiku mwangi,mwangi rosemary wanjiku,0,4.1564487302049126e-05,4.1564487302049126e-05,1.0,1.0,0
2,11.685238932999157,0.9996964286965933,__splink__input_table_0,__splink__input_table_1,156,5174,TT23306Z9GPL,TT23306Z9GPL,4,0.00012506774502855713,...,0.9972173870636092,1.0,tt23306z9gpl marianne m irungu,irungu marianne mutunge,0,4.1564487302049126e-05,0.0001662579492081965,1.0,1.0,0
3,12.772423645963242,0.9998570924095064,__splink__input_table_0,__splink__input_table_1,141,5353,FT233060HJ9C,FT233060HJ9C,4,8.337849668570476e-05,...,1.85241107447845,0.5083293252807716,icea policy ref jonathan katua muteta,ddi nic bank,0,4.1564487302049126e-05,0.25587098383141443,1.0,1.0,0
4,18.454247685936988,0.9999972156957525,__splink__input_table_0,__splink__input_table_1,160,5354,FT23306VV967,FT23306VV967,4,8.337849668570476e-05,...,1.85241107447845,26.094238697746277,1caroline chepngetich tegek rfbpolicy no 02011123015075,ddi nic bank,0,4.1564487302049126e-05,0.25587098383141443,1.0,1.0,0


In [ ]:
clusters = linker_detailed.clustering.cluster_pairwise_predictions_at_threshold(
    predictions, threshold_match_probability=0.60
)
df_clusters = clusters.as_pandas_dataframe().sort_values("cluster_id")
df_clusters.groupby("cluster_id").size().value_counts

INFO:splink.internals.connected_components:Completed iteration 1, root rows count 1
INFO:splink.internals.connected_components:Completed iteration 2, root rows count 0


<bound method IndexOpsMixin.value_counts of cluster_id
__splink__input_table_0-__-0       2
__splink__input_table_0-__-1       1
__splink__input_table_0-__-10      1
__splink__input_table_0-__-100     1
__splink__input_table_0-__-1000    1
                                  ..
__splink__input_table_1-__-9995    1
__splink__input_table_1-__-9996    1
__splink__input_table_1-__-9997    1
__splink__input_table_1-__-9998    1
__splink__input_table_1-__-9999    1
Length: 23511, dtype: int64>

In [ ]:
#records_to_view = df_predictions.as_record_dict(limit=5)
records_to_view = df_predictions.to_dict(orient='records')[:5]  # Convert DataFrame to a list of dictionaries#
linker_detailed.visualisations.waterfall_chart(records_to_view, filter_nulls=False)

alt.LayerChart(...)

In [ ]:
df_predictions.shape

(596, 46)

In [ ]:
df_predictions["match_key"].value_counts()

,count
match_key,
0,509
3,33
2,30
1,24


In [ ]:
df_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 46 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   match_weight               596 non-null    float64
 1   match_probability          596 non-null    float64
 2   source_dataset_l           596 non-null    object 
 3   source_dataset_r           596 non-null    object 
 4   unique_id_l                596 non-null    int64  
 5   unique_id_r                596 non-null    int64  
 6   trans_id_l                 596 non-null    object 
 7   trans_id_r                 576 non-null    object 
 8   gamma_trans_id             596 non-null    int32  
 9   tf_trans_id_l              596 non-null    float64
 10  tf_trans_id_r              576 non-null    float64
 11  bf_trans_id                596 non-null    float64
 12  bf_tf_adj_trans_id         596 non-null    float64
 13  m_batch_bulk_id_l          596 non-null    object 

In [ ]:
df_matches=df_predictions[["match_weight","match_probability","unique_id_l","unique_id_r",\
                           "trans_id_l","trans_id_r","gamma_trans_id",\
                          "m_batch_bulk_id_l","tf_m_batch_bulk_id_r","gamma_m_batch_bulk_id",\
                           "tf_policy_no_l","tf_policy_no_r","gamma_policy_no",\
                           "trx_date_l","trx_date_r" ,"trans_detail_l",\
                           "trans_detail_r","gamma_trans_detail",\
                           "trx_amount_r","trx_amount_l","gamma_trx_amount","match_key"]]

In [ ]:
# @title Match bank transactions with matched records

bank_df4_matched=bank_df4.merge(df_matches,how="left",left_on="index_bank",right_on="unique_id_l")
bank_df4_matched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3227 entries, 0 to 3226
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   index_bank             3227 non-null   int64         
 1   date                   3227 non-null   datetime64[ns]
 2   transaction_details_1  3227 non-null   object        
 3   transaction_details_2  3227 non-null   object        
 4   amount                 3227 non-null   float64       
 5   trx_type               3227 non-null   object        
 6   trans_type             3227 non-null   object        
 7   trx_detail             3227 non-null   object        
 8   period                 3227 non-null   period[Q-DEC] 
 9   merged_trans_id        3223 non-null   object        
 10  charge_id              237 non-null    object        
 11  batch_no               266 non-null    object        
 12  inhouse_cheque_no      30 non-null     object        
 13  che


\

# Output

In [ ]:
# @title Save results
!rm -rf {OUTPUT_DIR} && mkdir -p {OUTPUT_DIR}
suffix = "splink"
dest_path = OUTPUT_DIR/ f"{ACC_NAME}_recon_{suffix}_fuzzy.xlsx"
dest_path.parent.mkdir(parents=True, exist_ok=True)
results = {
    "bank_recon_fuzzy": df_matches,
    "bank_recon_com": bank_df4_matched,
}
with pd.ExcelWriter(dest_path) as writer:
    for name, data in results.items():
        data.to_excel(writer, sheet_name=name, index=False)

In [ ]:
# @title Copy the results to Google Drive
!mkdir -p {DATA_DIR}
!cp -r {OUTPUT_DIR} {DATA_DIR}